### NewsAPI.org

In [ ]:
pip install newsapi-python

In [ ]:
from newsapi import NewsApiClient
import pandas as pd

# Init
with open("NewsAPI_key.json") as infile:
    json_obj = json.load(infile)
    key =json_obj["key"]
    newsapi = NewsApiClient(api_key=key)

# Dummy list to store all articles
all_articles_list = []

# Loop through each word and fetch articles
for athlete in athletes:
    for page in range(1, 6):  # Loop though pages
        articles = newsapi.get_everything(q=athlete,
                                          sort_by='relevancy',
                                          page=page)
        # Extract the articles and add a new column for the search term
        if 'articles' in articles:
            for article in articles['articles']:
                article['athletes'] = athlete
                all_articles_list.append(article)
        else:
            break  # Exit the loop if no more articles are returned

# Convert the list of articles into a DataFrame
df = pd.DataFrame(all_articles_list)

encoded = json.dumps(all_articles_list)
all_articles_list = json.loads(encoded)

### Google News API

In [ ]:
#401 =not found
import os
import urllib.parse
import requests

# Replace with your actual API key
API_KEY = os.environ.get("API_KEY")
# Set the API key in headers
headers = {"apikey": API_KEY}

# Loop through each athlete and fetch news articles
for athlete in athletes:
    # Format the query
    query = {"q": athlete}
    
    # Build the URL to make the request
    url = f"https://api.serply.io/v1/news?" + urllib.parse.urlencode(query)
    
    # Make the request
    resp = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if resp.status_code == 200:
        results = resp.json()
        
        # Print the results
        print(f"Results for {athlete}:")
        for article in results.get('articles', []):
            print("Title:", article['title'])
            print("Description:", article['description'])
            print("URL:", article['url'])
            print()
    else:
        print(f"Failed to fetch results for {athlete}: {resp.status_code}")

## Articles Database json Agathe

I didn't know what the articles were about, it seems that it is about the NFL... I still need to perform some more analysis.

In [ ]:
import json

# Define the path to the JSON file
file_path = 'data-articles/news_0000002.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Extracting the necessary details from the JSON data
article_text = data["text"]

# Print the extracted details
print("Article Text:", article_text)


In [ ]:
import json
# Loop through files from 1 to 100
for i in range(1, 101):
    # Generate the file name with leading zeros
    file_path = f'data-articles/news_{i:07d}.json'
    
    try:
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extracting the necessary details from the JSON data
        article_text = data["text"]
        
        # Print the extracted details
        print(f"Article {i} Text:", article_text)
        print("------------------------------------------------------------")
    
    except FileNotFoundError:
        print(f"File {file_path} not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file {file_path}.")



Let's see what sports these articles are about.

In [ ]:
## Highlight the sport of each article - put it it the same of the json file

# Define the sports keywords to look for
sports_keywords = ["athletic", "NFL", "baseball"]

# Directory containing the JSON files
input_directory_path = 'data-articles'

# Directory to save the filtered articles
output_directory_path = 'data-articles/filtered-articles'
os.makedirs(output_directory_path, exist_ok=True)

# Loop through files from 1 to 100
for i in range(1, 101):
    # Generate the input file name with leading zeros
    input_file_path = os.path.join(input_directory_path, f'news_{i:07d}.json')
    
    try:
        # Open and read the JSON file
        with open(input_file_path, 'r') as file:
            data = json.load(file)
        
        # Extract the text of the article
        article_text = data.get("text", "")
        
        # Check if any of the sports keywords are in the article text
        found_keywords = [keyword for keyword in sports_keywords if keyword.lower() in article_text.lower()]
        if found_keywords:
            # Print the found keywords
            print(f"Article {i} mentions the following sports keywords: {', '.join(found_keywords)}")
            
            # Save only the text to a new JSON file in the output directory
            output_file_path = os.path.join(output_directory_path, f'filtered_{i:07d}_{found_keywords[0]}.json')
            with open(output_file_path, 'w') as output_file:
                json.dump({"text": article_text}, output_file, indent=4)
            print(f"Article {i} text has been saved to {output_file_path}.")
    
    except FileNotFoundError:
        print(f"File {input_file_path} not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON in file {input_file_path}.")

So as you can see, we have mostly baseball and NFL articles....

Let's find something else.

## Get articles from google - Agathe
Webscraping (not API)

In [ ]:
pip install googlesearch-python requests beautifulsoup4

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import os
import time

In [ ]:
## Find articles from a google search - RANDOM SPORTS article for nom

def fetch_wikipedia_articles(query, num_results=10):
    # Perform Google search limited to Wikipedia
    search_results = search(query + " site:en.wikipedia.org", num_results=num_results, lang="en")
    
    articles = []

    for url in search_results:
        try:
            # Make a request to the URL
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract the title
            title = soup.find('title').text
            
            # Extract the content
            content = ''
            for paragraph in soup.find_all('p'):
                content += paragraph.text + '\n'
            
            # Save the article
            articles.append({
                'url': url,
                'title': title,
                'content': content
            })
        
        except requests.RequestException as e:
            print(f"Failed to fetch {url}: {e}")
    
    return articles

def save_articles(articles, folder_name):
    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    for idx, article in enumerate(articles, start=1):
        filename = os.path.join(folder_name, f"article_{idx}.txt")
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(f"Title: {article['title']}\n")
            file.write(f"URL: {article['url']}\n")
            file.write("Content:\n")
            file.write(article['content'])

if __name__ == "__main__":
    # Example queries
    queries = ["athletics performance", "gymnastics"]
    
    all_articles = {}
    
    for query in queries:
        print(f"Fetching Wikipedia articles for: {query}")
        articles = fetch_wikipedia_articles(query)
        all_articles[query] = articles
        
    for query, articles in all_articles.items():
        print(f"\nWikipedia Articles for query: {query}")
        save_articles(articles, "google-api-data")
        print("Articles saved.")

### BE CAREFUL
# You can get an error if you run too often this cell!!



Now we want to get the articles of athletes. Let's first look at the 5000m 10 best athletes.
Yomif KEJELCHA, Hagos GEBRHIWET, Berihu AREGAWI, Telahun Haile BEKELE, Jakob INGEBRIGTSEN, Jacob KIPLIMO, Selemon BAREGA,Grant FISHER,Luis GRIJALVA,Joshua CHEPTEGEI

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import os

def fetch_articles_for_athlete(athlete_name, num_results=10):
    query = f"{athlete_name} 5000m athlete"
    search_results = search(query, num_results=num_results, lang="en")

    articles = []

    for url in search_results:
        try:
            # Make a request to the URL
            response = requests.get(url)
            response.raise_for_status()

            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the title
            title = soup.find('title').text

            # Extract the content
            content = ''
            for paragraph in soup.find_all('p'):
                content += paragraph.text + '\n'

            # Save the article
            articles.append({
                'url': url,
                'title': title,
                'content': content
            })

        except requests.HTTPError as e:
            print(f"Failed to fetch {url}: {e}")
        except Exception as e:
            print(f"Error processing {url}: {e}")

    return articles

def save_articles_for_athlete(athlete_name, articles, folder_name):
    # Create a folder for the athlete if it doesn't exist
    athlete_folder = os.path.join(folder_name, athlete_name.replace(" ", "_"))
    if not os.path.exists(athlete_folder):
        os.makedirs(athlete_folder)

    # Save each article into a separate text file
    for idx, article in enumerate(articles, start=1):
        filename = os.path.join(athlete_folder, f"article_{idx}.txt")
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(f"Title: {article['title']}\n")
            file.write(f"URL: {article['url']}\n")
            file.write("Content:\n")
            file.write(article['content'])

if __name__ == "__main__":
    # List of athletes
    athletes = [
        "Yomif KEJELCHA", "Hagos GEBRHIWET", "Berihu AREGAWI", "Telahun Haile BEKELE",
        "Jakob INGEBRIGTSEN", "Jacob KIPLIMO", "Selemon BAREGA", "Grant FISHER",
        "Luis GRIJALVA", "Joshua CHEPTEGEI"
    ]

    # Number of articles to fetch for each athlete
    num_articles_per_athlete = 10

    for athlete in athletes:
        print(f"Fetching articles for: {athlete}")
        articles = fetch_articles_for_athlete(athlete, num_results=num_articles_per_athlete)
        save_articles_for_athlete(athlete, articles, "google-api-data")
        print(f"Articles for {athlete} saved.")


**From ESPN**

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import os

def fetch_articles_for_athlete_from_espn(athlete_name, num_results=10):
    query = f"{athlete_name} site:espn.com"
    search_results = search(query, num_results=num_results, lang="en")

    articles = []

    for url in search_results:
        try:
            # Make a request to the URL
            response = requests.get(url)
            response.raise_for_status()

            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the title
            title = soup.find('title').text

            # Extract the content
            content = ''
            for paragraph in soup.find_all('p'):
                content += paragraph.text + '\n'

            # Save the article
            articles.append({
                'url': url,
                'title': title,
                'content': content
            })

        except requests.HTTPError as e:
            print(f"Failed to fetch {url}: {e}")
        except Exception as e:
            print(f"Error processing {url}: {e}")

    return articles

if __name__ == "__main__":
    # List of athletes
    athletes = [
        "Yomif KEJELCHA", "Hagos GEBRHIWET", "Berihu AREGAWI", "Telahun Haile BEKELE",
        "Jakob INGEBRIGTSEN", "Jacob KIPLIMO", "Selemon BAREGA", "Grant FISHER",
        "Luis GRIJALVA", "Joshua CHEPTEGEI"
    ]

    # Number of articles to fetch for each athlete
    num_articles_per_athlete = 10

    for athlete in athletes:
        print(f"Fetching articles for: {athlete}")
        articles = fetch_articles_for_athlete_from_espn(athlete, num_results=num_articles_per_athlete)
        save_articles_for_athlete(athlete, articles, "google-api-data")
        print(f"Articles for {athlete} saved.")


# Prompt engineering Test

In [ ]:
pip install transformers

In order to run this code, I need to use a virtual environment (where ten

In [ ]:
# Step 1: Load pre-trained text classification model
from transformers import pipeline

classifier = pipeline("text-classification")

# Step 2: Define your articles
articles = [
    "Article 1 text...",
    "Article 2 text...",
    "Article 3 text...",
    # Add more articles as needed
]

# Step 3: Active learning loop
labeled_data = []
unlabeled_data = articles.copy()

while len(unlabeled_data) > 0:
    # Step 3a: Predict labels for unlabeled data
    predictions = classifier(unlabeled_data)
    
    # Step 3b: Select a subset of predictions for manual verification
    for article, prediction in zip(unlabeled_data, predictions):
        print("Article:", article)
        print("Predicted Label:", prediction['label'])
        print("Confidence:", prediction['score'])
        user_input = input("Enter the correct label (or skip): ")
        
        if user_input.lower() != "skip":
            labeled_data.append((article, user_input))
            unlabeled_data.remove(article)
    
    # Step 3c: Retrain the model on the expanded labeled dataset
    labeled_texts = [article for article, label in labeled_data]
    labels = [label for article, label in labeled_data]
    
    # Perform model retraining with labeled data (code not provided)
    # retrain_model(labeled_texts, labels)

# Final labeled dataset
print("Final labeled dataset:")
for article, label in labeled_data:
    print("Article:", article)
    print("Label:", label)
